In [46]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import math
import random
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [47]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # train data path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

### Dividing the features by the number of jets

In [48]:
# dividing the rows of tX by the number of jets, dropping the column Pri_Jet_Num and adding an extra column of np.ones
zero_indices = []
one_indices = []
two_three_indices = []
zero_indices = np.where(tX[:,22]==0)[0]
one_indices = np.where(tX[:,22]==1)[0]
two_three_indices = np.where(np.logical_or(tX[:,22]==2, tX[:,22]==3))[0]
tX_0 = tX[zero_indices, :]
tX_0 = np.delete(tX_0, 22, axis=1)
tX_1 = tX[one_indices, :]
tX_1 = np.delete(tX_1, 22, axis=1)
tX_2_3 = tX[two_three_indices, :]

### Dividing also the output by the type of particle

In [49]:
y_0 = y[zero_indices]
y_1 = y[one_indices]
y_2_3 = y[two_three_indices]

### Adding a column of zeros and ones to detect whether the mass has been measured or not

In [50]:
# take the indices where the mass is not calculated, add the column which has 0 in those indices
# and 1 everywhere else for all matrices 0,1,2_3
zero_indices_0 = np.where(tX_0[:,1] == -999.)[0]
column_to_add = np.array([0 if i in zero_indices_0 else 1 for i in range(tX_0.shape[0])])
tX_0 = np.insert(tX_0, 0, column_to_add, axis=1)
zero_indices_1 = np.where(tX_1[:,1] == -999.)[0]
column_to_add = np.array([0 if i in zero_indices_1 else 1 for i in range(tX_1.shape[0])])
tX_1 = np.insert(tX_1, 0, column_to_add, axis=1)
zero_indices_2_3 = np.where(tX_2_3[:,1] == -999.)[0]
column_to_add = np.array([0 if i in zero_indices_2_3 else 1 for i in range(tX_2_3.shape[0])])
tX_2_3 = np.insert(tX_2_3, 0, column_to_add, axis=1)

### Changing the labels to {0,1}

In [51]:
# y == 0 non detected Boson, y == 1 detected Boson
y_ = np.array([0 if l == -1 else 1 for l in y])

### Throwing away the outliers from the training data

In [52]:
for i in range(1, tX_2_3.shape[1]):
    index_column_valid =np.where(tX_2_3[:,i] != -999.)[0]
    column_25_quantile, column_75_quantile = np.quantile(tX_2_3[index_column_valid,i], 
                                                         np.array([0.25, 0.75]))
    interquantile = column_75_quantile-column_25_quantile
    column_15_quantile, column_85_quantile = np.quantile(tX_2_3[index_column_valid,i], 
                                                         np.array([0.15, 0.85]))
    indices_outliers = np.where((column_15_quantile - 1.5 * interquantile >= tX_2_3[index_column_valid,i])
                                             | (tX_2_3[index_column_valid,i] >= 
                                                column_85_quantile + 1.5 * interquantile))[0]
    #indices_outliers = np.argwhere((tX_tilda_2_3[index_column_valid,i] >= column_85_quantile + 1.5 * interquantile) | 
                                  #(column_15_quantile - 1.5 * interquantile >= tX_tilda_2_3[index_column_valid,i]))
    median = np.median(tX_2_3[index_column_valid, i], axis = 0)
    #print(np.sort(tX_tilda_2_3[index_column_valid[indices_outliers],i]).T)
    #print(np.where(tX_tilda_2_3[indices_outliers,i])==-999.)
    #print(median)
    tX_2_3[index_column_valid[indices_outliers],i] =  median

In [53]:
col_to_delete = []
for i in range(1, tX_0.shape[1]):
    index_column_valid =np.where(tX_0[:,i] != -999.)[0]
    if len(index_column_valid)==0:
        #we drop the column (we will have to do the same for the test set as well)
        col_to_delete.append(i)
    else :
        column_25_quantile, column_75_quantile = np.quantile(tX_0[index_column_valid,i], 
                                                         np.array([0.25, 0.75]))
        interquantile = column_75_quantile-column_25_quantile
        column_15_quantile, column_85_quantile = np.quantile(tX_0[index_column_valid,i], 
                                                         np.array([0.15, 0.85]))
        indices_outliers = np.where((column_15_quantile - 1.5 * interquantile >= tX_0[index_column_valid,i])
                                             | (tX_0[index_column_valid,i] >= 
                                                column_85_quantile + 1.5 * interquantile))[0]
        #indices_outliers = np.argwhere((tX_tilda_2_3[index_column_valid,i] >= column_85_quantile + 1.5 * interquantile) | 
                                  #(column_15_quantile - 1.5 * interquantile >= tX_tilda_2_3[index_column_valid,i]))
        median = np.median(tX_0[index_column_valid, i], axis = 0)
        #print(np.sort(tX_tilda_2_3[index_column_valid[indices_outliers],i]).T)
        #print(np.where(tX_tilda_2_3[indices_outliers,i])==-999.)
        #print(median)
        tX_0[index_column_valid[indices_outliers],i] =  median
tX_0 = np.delete(tX_0, col_to_delete, axis=1)
print(tX_0.shape)
print(col_to_delete)

(99913, 20)
[5, 6, 7, 13, 23, 24, 25, 26, 27, 28]


In [54]:
col_to_delete = []
for i in range(1, tX_1.shape[1]):
    index_column_valid =np.where(tX_1[:,i] != -999.)[0]
    if len(index_column_valid)==0:
        #we drop the column (we will have to do the same for the test set as well)
        col_to_delete.append(i)
    else :
        column_25_quantile, column_75_quantile = np.quantile(tX_1[index_column_valid,i], 
                                                         np.array([0.25, 0.75]))
        interquantile = column_75_quantile-column_25_quantile
        column_15_quantile, column_85_quantile = np.quantile(tX_1[index_column_valid,i], 
                                                         np.array([0.15, 0.85]))
        indices_outliers = np.where((column_15_quantile - 1.5 * interquantile >= tX_1[index_column_valid,i])
                                             | (tX_1[index_column_valid,i] >= 
                                                column_85_quantile + 1.5 * interquantile))[0]
        #indices_outliers = np.argwhere((tX_tilda_2_3[index_column_valid,i] >= column_85_quantile + 1.5 * interquantile) | 
                                  #(column_15_quantile - 1.5 * interquantile >= tX_tilda_2_3[index_column_valid,i]))
        median = np.median(tX_1[index_column_valid, i], axis = 0)
        #print(np.sort(tX_tilda_2_3[index_column_valid[indices_outliers],i]).T)
        #print(np.where(tX_tilda_2_3[indices_outliers,i])==-999.)
        #print(median)
        tX_1[index_column_valid[indices_outliers],i] =  median
tX_1 = np.delete(tX_1, col_to_delete, axis=1)
print(col_to_delete)

[5, 6, 7, 13, 26, 27, 28]


### Now we substitute the -999 values with the median

In [55]:
for i in range(1, tX_2_3.shape[1]):
    index_column_non_valid =np.where(tX_2_3[:,i] == -999.)[0]
    index_column_valid =np.where(tX_2_3[:,i] != -999.)[0]
    median = np.median(tX_2_3[index_column_valid, i], axis = 0)
    tX_2_3[index_column_non_valid,i] =  median

In [56]:
for i in range(1, tX_1.shape[1]):
    index_column_non_valid =np.where(tX_1[:,i] == -999.)[0]
    index_column_valid =np.where(tX_1[:,i] != -999.)[0]
    median = np.median(tX_1[index_column_valid, i], axis = 0)
    tX_1[index_column_non_valid,i] =  median

In [57]:
for i in range(1, tX_0.shape[1]):
    index_column_non_valid =np.where(tX_0[:,i] == -999.)[0]
    index_column_valid =np.where(tX_0[:,i] != -999.)[0]
    median = np.median(tX_0[index_column_valid, i], axis = 0)
    tX_0[index_column_non_valid,i] =  median

### Now we standardize the data

In [58]:
tX_2_3[:,1:],_,_ = standardize(tX_2_3[:,1:]) #we standardize everything a part from the column added manually

In [59]:
print(tX_2_3)
print(np.count_nonzero(tX_2_3 == -999.))

[[ 1.          0.97351249  0.46588281 ...  0.61614788 -1.36131161
  -0.70374641]
 [ 1.         -0.82851663 -0.77157038 ...  0.11608109  1.71034105
   0.2995537 ]
 [ 1.          1.3538447  -0.27431587 ...  0.07030726 -1.52202162
   0.12704911]
 ...
 [ 1.          0.04006392  0.02513449 ...  0.25930888  0.22982758
   0.42357227]
 [ 1.          0.66304099 -1.0843679  ...  0.29031696 -1.21821366
  -0.20699627]
 [ 1.          0.04006392  0.31977857 ... -0.02271698 -0.62490751
   0.05569682]]
0


In [60]:
print(tX_0[:,-1])

[0. 0. 0. ... 0. 0. 0.]


In [61]:
tX_0[:,1:-1],_,_ = standardize(tX_0[:,1:-1]) 
# the last column is not standardized since it is the zero vector (to be dropped?)

In [62]:
print(tX_0)

[[ 1.          1.05744907  0.7827665  ...  0.04662815 -0.7724943
   0.        ]
 [ 1.          2.14505538 -1.37519852 ... -0.4674532  -1.44727052
   0.        ]
 [ 1.         -0.24632406 -0.2496121  ...  0.0267496   0.12380588
   0.        ]
 ...
 [ 1.         -0.0469687   0.00532098 ... -0.46524447 -0.8883482
   0.        ]
 [ 1.         -0.60851918 -1.29333222 ...  0.46131675 -0.22629665
   0.        ]
 [ 1.         -0.0469687   0.49300595 ... -0.86778508 -0.49908812
   0.        ]]


In [63]:
tX_1[:,1:],_,_ = standardize(tX_1[:,1:])

In [64]:
print(tX_1)

[[ 1.00000000e+00  1.55539992e+00  7.27047143e-01 ...  3.98445313e-01
   6.45414781e-01 -4.14297220e-01]
 [ 1.00000000e+00  1.45598722e-03  3.58462301e+00 ...  1.12748232e+00
  -1.10752634e+00 -4.89864560e-01]
 [ 1.00000000e+00  1.36261180e+00 -1.05809645e+00 ... -3.92076740e-01
  -9.40265168e-01 -1.01072441e+00]
 ...
 [ 1.00000000e+00  1.45598722e-03  1.01732036e+00 ... -4.67286130e-01
  -3.80160315e-01  8.39087556e-01]
 [ 1.00000000e+00  6.75509966e-01  9.95415259e-01 ... -6.76994064e-01
   1.39533906e+00  5.32418071e-01]
 [ 1.00000000e+00 -2.21030015e-01  4.74893699e-01 ...  9.88591985e-01
  -8.30516498e-02 -5.76298292e-01]]


### We insert the column for the bias term

In [65]:
tX_tilda_0 = np.insert(tX_0, 0, np.ones(tX_0.shape[0]), axis=1)
tX_tilda_1 = np.insert(tX_1, 0, np.ones(tX_1.shape[0]), axis=1)
tX_tilda_2_3 = np.insert(tX_2_3, 0, np.ones(tX_2_3.shape[0]), axis=1)

In [66]:
# colors = ['red', 'blue']
# x_pos=[]
# x_neg=[]

# for j in range(len(y)):
#  if(y[j]==1):
#       x_pos.insert(0,tX[j])
#    else:
#        x_neg.insert(0,tX[j])
# xpos = np.array(x_pos)
# xneg = np.array(x_neg)
# for i in range(tX.shape[1]):
#  plt.hist(xpos[:,i], alpha = 0.5, color = 'r', bins = 100)
#  plt.hist(xneg[:,i], alpha = 0.5, color = 'b', bins = 100)
#  plt.show()

## Do your thing crazy machine learning thing here :) ...

In [67]:
def compute_loss(y, tx, w):
    N = y.shape[0]
    e = y - tx @ w 
    loss = 1/(2*N) * np.dot(e,e)
    return loss

In [68]:
def compute_gradient(y, tx, w):
    N = y.shape[0]
    e = y - tx @ w
    gradient = -(1/N) * (tx.T) @ (e)
    return gradient

In [69]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        loss = compute_loss(y,tx,w)
        gradient = compute_gradient(y,tx,w)
        w = w - gamma * gradient
        ws.append(w)
        losses.append(loss)
        # print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
        # bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return losses, ws

In [70]:
def compute_stoch_gradient(y, tx, w):
    N = y.shape[0]
    random_number = random.randint(0,N)
    #random_number =1
    xn = tx[random_number,:]
    random_gradient = - np.dot(xn, y[random_number] - np.dot(xn,w))
    return random_gradient

In [71]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        loss = compute_loss(y,tx,w)
        stoch_gradient = compute_stoch_gradient(y,tx,w)
        w = w - gamma * stoch_gradient
        ws.append(w)
        losses.append(loss)
        # print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
        #    bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return losses, ws

In [72]:
from proj1_helpers import *

def least_squares(y, tx):
    """calculate the least squares solution."""
    forcing_term = np.transpose(tx) @ y
    coefficient_matrix = np.transpose(tx) @ tx
    w = np.linalg.solve(coefficient_matrix, forcing_term)
    return w

def test_your_least_squares(y, tx):
    """compare the solution of the normal equations with the weights returned by gradient descent algorithm."""
    w_least_squares = least_squares(y, tx)
    initial_w = np.zeros(tx.shape[1])
    max_iters = 50
    gamma = 0.7
    losses_gradient_descent, w_gradient_descent = gradient_descent(y, tx, initial_w, max_iters, gamma)
    w = w_gradient_descent[-1]
    err = np.linalg.norm(w_least_squares-w)
    return err

In [73]:
def ridge_regression(y, tx, lambda_):
    """implement ridge regression."""
    N = tx.shape
    lambda_prime = 2 * N[0] * lambda_
    coefficient_matrix = np.transpose(tx) @ tx + lambda_prime * np.eye(N[1])
    forcing_term = np.transpose(tx) @ y
    w = np.linalg.solve(coefficient_matrix, forcing_term)
    return w

def debug_ridge(y, tx):
    """debugging the ridge regression by setting lambda=0."""
    w_least_squares = least_squares(y, tx)
    w_0 = ridge_regression(y, tx, 0)
    err = np.linalg.norm(w_least_squares-w_0)
    return err

In [74]:
def sigmoid(t):
    """apply the sigmoid function on t."""
    return np.exp(t) / (1+np.exp(t))

In [75]:
def calculate_loss(y, tx, w):
    """compute the loss: negative log likelihood."""
    N = y.shape[0]
    e = - (y*np.log(sigmoid(tx @ w)) +
                  (1-y)*np.log(1-sigmoid(tx @ w)))
    return e.sum()

In [76]:
def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    return np.transpose(tx) @ (sigmoid(tx @ w) - y)

In [77]:
def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descent using logistic regression.
    Return the loss and the updated w.
    """
    loss = calculate_loss(y, tx, w)
    grad = calculate_gradient(y, tx, w)
    w = w - gamma * grad
    return loss, w

In [78]:
def calculate_hessian(y, tx, w):
    """return the Hessian of the loss function."""
    diag = sigmoid(tx @ w) * (1 - sigmoid(tx @ w))
    D = diag * np.eye(tx.shape[0])
    return np.transpose(tx) @ D @ tx

In [79]:
def logistic_regression(y, tx, w):
    """return the loss, gradient, and Hessian."""
    grad = calculate_gradient(y, tx, w)
    hess = calculate_hessian(y, tx, w)
    loss = calculate_loss(y, tx, w)
    return loss, grad, hess

In [80]:
def learning_by_newton_method(y, tx, w, gamma):
    """
    Do one step on Newton's method.
    return the loss and updated w.
    """
    loss, grad, hess = logistic_regression(y, tx, w)
    sol = np.linalg.solve(hess, grad)
    w = w - gamma * sol
    return loss, w

In [81]:
def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss, gradient"""
    loss = calculate_loss(y, tx, w) + lambda_*np.linalg.norm(w) ** 2
    grad = calculate_gradient(y, tx, w) + 2*lambda_*w
    hess = calculate_hessian(y, tx, w) + 2*lambda_*np.eye(w.shape[0])
    return loss, grad, hess

In [82]:
def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.
    """
    loss, grad, hess = penalized_logistic_regression(y, tx, w, lambda_)
    sol = np.linalg.solve(hess, grad)
    w = w - gamma * sol
    return loss, w

In [83]:
def build_k_indices(y, k_fold, seed):
    N = y.shape[0]
    np.random.seed(seed)
    interval = int(np.floor(N / k_fold))
    indices = np.random.permutation(N)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

In [84]:
def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression."""
    N = y.shape[0]
    k_fold = k_indices.shape[0]
    list_ = []
    interval = int(N/k_fold)
    for i in range(k_fold):
        if i != k:
            list_.append(i)
    x_training = np.zeros(int((k_fold-1)/k_fold*N))
    y_training = np.zeros(int((k_fold-1)/k_fold*N))
    for j in range(len(list_)):
        x_training[interval*(j):interval*(j+1)] = x[np.array([k_indices[list_[j]]])]
    x_testing = x[k_indices[k]]
    for j in range(len(list_)):
        y_training[interval*(j):interval*(j+1)] = y[np.array([k_indices[list_[j]]])]
    y_testing = y[k_indices[k]]
    x_training_augmented = build_poly(x_training, degree)
    x_testing_augmented = build_poly(x_testing, degree)
    w_opt_training = ridge_regression(y_training, x_training_augmented, lambda_)
    loss_tr = compute_mse(y_training, x_training_augmented, w_opt_training)
    loss_te = compute_mse(y_testing, x_testing_augmented, w_opt_training)
    return loss_tr, loss_te

### Trial of different models on the three training matrices

### Generate predictions and save ouput in csv format for submission:

In [41]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [42]:
def predict_labels(weights, tX_test):
    y = np.array(tX_test) @ np.array(weights)
    labels = [1 if l > 0 else -1 for l in y]
    return labels

In [43]:
# indeces_lepton = [0, 1, 2, 3, 7, 8, 9, 10, 11, 12, 16, 17, 18, 21, 22]
# indeces_hadronic_tau = [0, 3, 7, 8, 9, 10, 11, 13, 14, 15, 21, 22]
# indeces_jet = [0, 4, 5, 6, 8, 9, 12, 21, 22, 23, 24, 25, 26, 27, 28, 29]
# indeces_MTE = [0, 1, 3, 8, 9, 11, 19, 20, 21, 22]

# tX_lepton_test = tX_test[:, indeces_lepton]
# tX_tilda_lepton_test = np.insert(tX_lepton_test, 0, np.ones(tX_test.shape[0]), axis=1)
# labels_lepton = predict_labels(w_opt_lepton, tX_tilda_lepton_test)
# print(1 / np.array(labels_lepton).shape[0] * np.count_nonzero(np.array(labels_lepton) == 1))

# tX_hadronic_tau_test = tX_test[:, indeces_hadronic_tau]
# tX_tilda_hadronic_tau_test = np.insert(tX_hadronic_tau_test, 0, np.ones(tX_test.shape[0]), axis=1)
# labels_hadronic_tau = predict_labels(w_opt_hadronic_tau, tX_tilda_hadronic_tau_test)
# print(1 / np.array(labels_hadronic_tau).shape[0] * np.count_nonzero(np.array(labels_hadronic_tau) == 1))

# tX_jet_test = tX_test[:, indeces_jet]
# tX_tilda_jet_test = np.insert(tX_jet_test, 0, np.ones(tX_test.shape[0]), axis=1)
# labels_jet = predict_labels(w_opt_jet, tX_tilda_jet_test)
# print(1 / np.array(labels_jet).shape[0] * np.count_nonzero(np.array(labels_jet) == 1))

# tX_MTE_test = tX_test[:, indeces_MTE]
# tX_tilda_MTE_test = np.insert(tX_MTE_test, 0, np.ones(tX_test.shape[0]), axis=1)
# labels_MTE = predict_labels(w_opt_MTE, tX_tilda_MTE_test)
# print(1 / np.array(labels_MTE).shape[0] * np.count_nonzero(np.array(labels_MTE) == 1))

In [44]:
# TRIAL, UNDERESTIMATION OF THE TRUE PROBABILITY TO GET A BOSON
# count = np.array(labels_MTE) + np.array(labels_lepton) + np.array(labels_hadronic_tau) + np.array(labels_jet)
# TrueSignal = np.array([int(bool((counted == 4))) for counted in count])
# print( 1 / np.array(count).shape[0] * np.count_nonzero(np.array(TrueSignal) == 1))

In [45]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

NameError: name 'weights' is not defined